In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/generated/stop_times_passengers_5days.csv')

In [3]:
df.head()

,Unnamed: 0,stop_id,route_short_name,arrival_time,departure_time,trip_id,stop_sequence,hospital,school,passengers,passengers_out
0,0,3049,L1,06:32:00,06:32:00,0004187192024090920250601,1,0,0,14,7
1,1,3050,L1,06:32:00,06:32:00,0004187192024090920250601,2,0,0,4,2
2,2,3052,L1,06:33:00,06:33:00,0004187192024090920250601,3,0,0,14,1
3,3,3054,L1,06:34:00,06:34:00,0004187192024090920250601,4,0,0,7,4
4,4,3047,L1,06:35:00,06:35:00,0004187192024090920250601,5,0,0,8,5


In [16]:
df_cleaned = df.drop(labels=['Unnamed: 0'], axis=1)

In [17]:
df_cleaned['passengers_out'] = df_cleaned['passengers_out'] + 7

In [18]:
df_cleaned.groupby('trip_id').agg('sum')[['passengers', 'passengers_out']]

,passengers,passengers_out
trip_id,,
0004187192024090920250601,47,54
0004187192024090920250602,64,54
0004187192024090920250603,58,47
0004187192024090920250604,80,61
0004187192024090920250605,58,42
...,...,...
0004254132025033120250601,201,148
0004254132025033120250602,186,130
0004254132025033120250603,209,132


In [ ]:
X = df_cleaned[]